In [1]:
 
'''Construct an FP-tree using suitable programming language for appropriate data set for association rule mining. 
#Explain all the steps of the tree construction and draw the resulting tree.
#(Minimum support count threshold for association rules [default: 2]) Based on this tree answer the questions: 
# a) Find maximum frequent itemset.
# b) How many transactions does it contain? 
#c) Simulate frequent pattern enumeration based on the FP-tree constructed. 
# d) Give comparative analysis of this process with Apriori algorithm.'''
from collections import defaultdict, Counter
from itertools import combinations

class FPNode:
    def __init__(self, item, count, parent):
        self.item = item
        self.count = count
        self.parent = parent
        self.children = {}
        self.link = None

class FPTree:
    def __init__(self, transactions, min_support):
        self.min_support = min_support
        self.header_table = {}
        self.root = FPNode(None, 1, None)
        self.freq_items = {}
        self.build_header_table(transactions)
        self.build_tree(transactions)

    def build_header_table(self, transactions):
        freq = defaultdict(int)
        for trans in transactions:
            for item in trans:
                freq[item] += 1
        self.freq_items = {item: count for item, count in freq.items() if count >= self.min_support}
        self.freq_items = dict(sorted(self.freq_items.items(), key=lambda x: (-x[1], x[0])))

    def sort_items(self, transaction):
        return [item for item in sorted(transaction, key=lambda x: (-self.freq_items.get(x, 0))) if item in self.freq_items]

    def build_tree(self, transactions):
        for trans in transactions:
            sorted_items = self.sort_items(trans)
            self.insert_tree(sorted_items, self.root)

    def insert_tree(self, items, node):
        if not items:
            return
        first = items[0]
        if first in node.children:
            node.children[first].count += 1
        else:
            new_node = FPNode(first, 1, node)
            node.children[first] = new_node
            if first in self.header_table:
                current = self.header_table[first]
                while current.link:
                    current = current.link
                current.link = new_node
            else:
                self.header_table[first] = new_node
        self.insert_tree(items[1:], node.children[first])

    def display(self, node=None, indent=0):
        if node is None:
            node = self.root
        for child in node.children.values():
            print('  ' * indent + f"{child.item} ({child.count})")
            self.display(child, indent + 1)

    def mine_patterns(self):
        patterns = defaultdict(int)
        items = list(self.freq_items.keys())[::-1]
        for item in items:
            conditional_patterns = []
            node = self.header_table[item]
            while node:
                path = []
                parent = node.parent
                while parent and parent.item:
                    path.append(parent.item)
                    parent = parent.parent
                for _ in range(node.count):
                    conditional_patterns.append(path[::-1])
                node = node.link

            # Count conditional pattern combinations
            count = defaultdict(int)
            for pattern in conditional_patterns:
                for i in range(1, len(pattern) + 1):
                    for subset in combinations(pattern, i):
                        count[tuple(sorted(subset + (item,)))] += 1
            for pattern, cnt in count.items():
                if cnt >= self.min_support:
                    patterns[pattern] = cnt
        return patterns

# Get user input
def get_user_input():
    print("Enter number of transactions:")
    n = int(input())
    transactions = []
    print("Enter transactions (items separated by spaces):")
    for i in range(n):
        trans = input(f"T{i+1}: ").strip().lower().split()
        transactions.append(trans)
    min_support = int(input("Enter minimum support count (default 2): ") or 2)
    return transactions, min_support

# Main Program
transactions, min_support = get_user_input()
tree = FPTree(transactions, min_support)
print("\n✅ FP-Tree:")
tree.display()

patterns = tree.mine_patterns()
print("\n✅ Frequent Itemsets (Support ≥", min_support, "):")
for pattern, count in sorted(patterns.items(), key=lambda x: (-len(x[0]), -x[1], x[0])):
    print(f"{set(pattern)} → {count}")

# a) Maximum frequent itemset
if patterns:
    max_len = max(len(p) for p in patterns)
    max_sets = [(p, c) for p, c in patterns.items() if len(p) == max_len]
    print("\n🔸 a) Maximum Frequent Itemsets:")
    for p, c in max_sets:
        print(f"{set(p)} → {c}")

    # b) Transactions it appears in
    print("\n🔸 b) Number of Transactions Containing Maximum Frequent Itemset(s):")
    for p, c in max_sets:
        print(f"{set(p)} appears in {c} transactions")
else:
    print("❌ No frequent itemsets found.")

# c) Pattern enumeration already printed

# d) Comparison with Apriori
print("\n🔸 d) FP-Growth vs Apriori:")
print("""
| Feature                  | FP-Growth                               | Apriori                                |
|--------------------------|-----------------------------------------|----------------------------------------|
| DB Scans                 | 2 (Build + Mining)                      | Multiple (one per k-itemset level)     |
| Candidate Generation     | ❌ No                                   | ✅ Yes                                  |
| Memory Usage             | Moderate to High (FP-tree in memory)    | Moderate                               |
| Performance on Dense Data| ✅ Efficient                            | ❌ Slower due to large candidate sets  |
| Approach                 | Pattern Growth                          | Join and Prune                         |
""")


Enter number of transactions:


 3


Enter transactions (items separated by spaces):


T1:  a b c d
T2:  a r c d
T3:  b d a m
Enter minimum support count (default 2):  2



✅ FP-Tree:
a (2)
  d (2)
    b (1)
      c (1)
    c (1)
d (1)
  a (1)
    b (1)

✅ Frequent Itemsets (Support ≥ 2 ):
{'a', 'b', 'd'} → 2
{'c', 'a', 'd'} → 2
{'a', 'b'} → 2
{'c', 'a'} → 2
{'a', 'd'} → 2
{'b', 'd'} → 2
{'c', 'd'} → 2

🔸 a) Maximum Frequent Itemsets:
{'c', 'a', 'd'} → 2
{'a', 'b', 'd'} → 2

🔸 b) Number of Transactions Containing Maximum Frequent Itemset(s):
{'c', 'a', 'd'} appears in 2 transactions
{'a', 'b', 'd'} appears in 2 transactions

🔸 d) FP-Growth vs Apriori:

| Feature                  | FP-Growth                               | Apriori                                |
|--------------------------|-----------------------------------------|----------------------------------------|
| DB Scans                 | 2 (Build + Mining)                      | Multiple (one per k-itemset level)     |
| Candidate Generation     | ❌ No                                   | ✅ Yes                                  |
| Memory Usage             | Moderate to High (FP-tree in memory